# Лаба 7. Построить content-based рекомендательную систему образовательных курсов

In [ ]:
import pandas as pd
import numpy as np
import json
import re

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

## Исходные данные

In [468]:
src_file = 'DO_record_per_line.json'

df = pd.read_json(src_file, lines=True)

In [469]:
df

,lang,name,cat,provider,id,desc
0,en,Accounting Cycle: The Foundation of Business M...,3/business_management|6/economics_finance,Canvas Network,4,This course introduces the basic financial sta...
1,en,American Counter Terrorism Law,11/law,Canvas Network,5,This online course will introduce you to Ameri...
2,fr,Arithmétique: en route pour la cryptographie,5/computer_science|15/mathematics_statistics_a...,Canvas Network,6,This course is taught in French Vous voulez co...
3,en,Becoming a Dynamic Educator,14/social_sciences,Canvas Network,7,We live in a digitally connected world. The wa...
4,en,Bioethics,2/biology_life_sciences,Canvas Network,8,This self-paced course is designed to show tha...
...,...,...,...,...,...,...
28148,en,Human Origins,2/biology_life_sciences,edX,28313,Explore the scientific evidence for human evol...
28149,en,Legal Risk Management Strategy for Multination...,,edX,28314,Examine the recent shift in the concept of cor...
28150,en,China (Part 1): The Political and Intellectual...,,edX,28315,"China’s past, present, and future: through his..."
28151,es,Entrepreneurship 102: ¿Que puedes hacer por tu...,,edX,28316,¿Que puedes hacer por tu cliente? No preguntes...


## courses to make recommendations

In [8]:
df_rec_for = pd.DataFrame(
    [
        [861, u'ru', u'\u0412\u0432\u0435\u0434\u0435\u043d\u0438\u0435 \u0432 \u0441\u043f\u0435\u0446\u0438\u0430\u043b\u044c\u043d\u043e\u0441\u0442\u044c \u043d\u0435\u0444\u0442\u044f\u043d\u0438\u043a\u0430'], 
        [4194, u'en', u'Excel VBA Tutorial - Excel Macro Tutorial'], 
        [8751, u'ru', u'\u0412\u0432\u0435\u0434\u0435\u043d\u0438\u0435 \u0432 \u0430\u043d\u0430\u043b\u0438\u0442\u0438\u043a\u0443 \u0431\u043e\u043b\u044c\u0448\u0438\u0445 \u043c\u0430\u0441\u0441\u0438\u0432\u043e\u0432 \u0434\u0430\u043d\u043d\u044b\u0445'], 
        [8974, u'es', u'After Effects Esencial by Txetxu de la Portilla'], 
        [9038, u'es', u'Aprende a Vender por Email by Lic. Leonardo Benijes'], 
        [19226, u'en', u'Requirements Analysis Destroys Ambiguity by Tom and Angela Hathaway']
    ],
    columns=['id','lang', 'name']
)

In [9]:
df_rec_for

,id,lang,name
0,861,ru,Введение в специальность нефтяника
1,4194,en,Excel VBA Tutorial - Excel Macro Tutorial
2,8751,ru,Введение в аналитику больших массивов данных
3,8974,es,After Effects Esencial by Txetxu de la Portilla
4,9038,es,Aprende a Vender por Email by Lic. Leonardo Be...
5,19226,en,Requirements Analysis Destroys Ambiguity by To...


Найдем номера строк где встречаются наши id

## Токенизируем и веторизируем тексты

### Стоп слова

А что нам мешает соединить вместе русские и английсие?

In [84]:
stopwords_rus = stopwords.words('russian')
stopwords_eng = stopwords.words('english')
stopwords_esp = stopwords.words('spanish')
stopwords_all = stopwords_rus + stopwords_eng + stopwords_esp

In [400]:
stopwords_dic = {
    'ru': stopwords.words('russian'),
    'en': stopwords.words('english'),
    'es': stopwords.words('spanish'),
}

### Токенайзер

Воспользуемся советом из лабы

In [25]:
def token_r(text):
    regex = re.compile(r'[\w\d]{2,}', re.U)
    return regex.findall(text.lower())

In [85]:
vectorizer = TfidfVectorizer(
    tokenizer=token_r, 
    stop_words=stopwords_all,
    ngram_range=(1, 2),
    max_features=10000
)

### TF-IDF для каждого языка по отдельности

In [86]:
lang_lst = {
    'ru',
    'en',
    'es'
}

In [95]:
tfidf_dic = dict((("tfidf_"+lang, vectorizer.fit_transform(df[df.lang == lang].desc)) for lang in lang_lst))

In [96]:
tfidf_dic

{'tfidf_ru': <1231x10000 sparse matrix of type '<class 'numpy.float64'>'
 	with 50809 stored elements in Compressed Sparse Row format>,
 'tfidf_en': <24553x10000 sparse matrix of type '<class 'numpy.float64'>'
 	with 2924562 stored elements in Compressed Sparse Row format>,
 'tfidf_es': <1374x10000 sparse matrix of type '<class 'numpy.float64'>'
 	with 219876 stored elements in Compressed Sparse Row format>}

## Найдем похожие курсы

In [180]:
cos_m_dic = {}
for lang, matrix in zip(lang_lst, tfidf_dic):
    # По номерам строк найдем курсы к которым подбираем рекомендации
    rec_id = df_rec_for[df_rec_for.lang == lang]['id'].tolist()
    df_lang = df[df.lang == lang].reset_index()
    rec_row_num = df_lang[df_lang.id.isin(rec_id)].index.values
    
    # добавим в словарь косинусную матрицу кажого языка
    cos_m_dic['cos_m_' + lang]  = cosine_similarity(tfidf_dic[matrix], tfidf_dic[matrix][rec_row_num])

In [181]:
cos_m_dic['cos_m_ru'].shape

(1231, 2)

## Объединим с основным датасетом

In [182]:
result_dic = {}

for lang, cos_m in zip(lang_lst, cos_m_dic):
    # Загоним матрицу в df
    df_cos = pd.DataFrame(cos_m_dic[cos_m])
    
    # Выделим основной датафрейм с нужным языком
    df_lang = df[df.lang == lang].reset_index()
    
    # Объединим с основным датафреймом
    result_dic['df_result' + lang] = df_lang.merge(df_cos, how='left', left_index=True, right_index=True)

## Выберем ТОП10

In [195]:
top_861 = (
        result_dic["df_resultru"]
            .query('id != 861')
            .sort_values(by=[0, 'name', 'id'], ascending=(False, True, True))
            .head(10)
            .id
            .tolist()
          )

top_8751 = (
        result_dic["df_resultru"]
            .query('id != 8751')
            .sort_values(by=[1, 'name', 'id'], ascending=(False, True, True))
            .head(10)
            .id
            .tolist()
          )

In [209]:
top_4194 = (
        result_dic["df_resulten"]
            .query('id != 4194')
            .sort_values(by=[0, 'name', 'id'], ascending=(False, True, True))
            .head(10)
            .id
            .tolist()
          )

top_19226 = (
        result_dic["df_resulten"]
            .query('id != 19226')
            .sort_values(by=[1, 'name', 'id'], ascending=(False, True, True))
            .head(10)
            .id
            .tolist()
          )

In [206]:
top_8974 = (
        result_dic["df_resultes"]
            .query('id != 8974')
            .sort_values(by=[0, 'name', 'id'], ascending=(False, True, True))
            .head(10)
            .id
            .tolist()
          )

top_9038 = (
        result_dic["df_resultes"]
            .query('id != 9038')
            .sort_values(by=[1, 'name', 'id'], ascending=(False, True, True))
            .head(10)
            .id
            .tolist()
          )

In [201]:
top_4194

[4631, 7042, 6918, 11170, 6669, 7392, 17975, 4712, 12105, 7103]

## Сформируем результат

In [210]:
output_dic = {
    "861": top_861,
    "8751": top_8751, 
    "4194": top_4194,
    "19226": top_19226,
    "8974": top_8974,
    "9038": top_9038,
}

In [211]:
output_dic

{'861': [13687, 937, 1070, 25922, 17128, 7132, 8092, 54, 932, 5221],
 '8751': [21899, 1416, 17066, 817, 17238, 886, 812, 877, 1400, 49],
 '4194': [4631, 7042, 6918, 11170, 6669, 7392, 17975, 4712, 12105, 7103],
 '19226': [19228, 19217, 9932, 13298, 25789, 22989, 6488, 24333, 12684, 11557],
 '8974': [12238, 7014, 10004, 5690, 6874, 7144, 5346, 7121, 16781, 12094],
 '9038': [24614, 17943, 23301, 18724, 9558, 21227, 10668, 24286, 17840, 19018]}

In [276]:
with open('../lab07.json', 'w') as outfile:
    json.dump(output_dic, outfile)

# Lab07s

In [430]:
df_rec_for2 = pd.DataFrame(
   [
       [3567, u'en', u'Adaptive Antennas and Phased Arrays'], 
       [4176, u'es', u'ERPs desde cero by Eugeni Vives'], 
       [11370, u'en', u'Marketing on YouTube'], 
       [21040, u'ru', u'\u0423\u043f\u0440\u0430\u0432\u043b\u0435\u043d\u0438\u0435 \xab\u0443\u043c\u043d\u044b\u043c \u0434\u043e\u043c\u043e\u043c\xbb'], 
       [22710, u'es', u'Aplicaciones creativas en el aula - Udemy'], 
       [26676, u'ru', u'\u0410\u043b\u0433\u043e\u0440\u0438\u0442\u043c\u0438\u0437\u0430\u0446\u0438\u044f. \u0412\u0432\u0435\u0434\u0435\u043d\u0438\u0435 \u0432 \u044f\u0437\u044b\u043a \u043f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u044f \u0421++']
   ],
    columns=['id','lang', 'name']
)

In [431]:
df_rec_for2

,id,lang,name
0,3567,en,Adaptive Antennas and Phased Arrays
1,4176,es,ERPs desde cero by Eugeni Vives
2,11370,en,Marketing on YouTube
3,21040,ru,Управление «умным домом»
4,22710,es,Aplicaciones creativas en el aula - Udemy
5,26676,ru,Алгоритмизация. Введение в язык программирован...


In [503]:
text = u'''
Escribir aplicaciones iOS y Android completamente funcionales utilizando las técnicas de vanguardia de 
PhoneGap Este curso de video le muestra cómo construir una aplicación móvil completamente funcional con 
PhoneGap de forma rápida y fácil. Comenzará instalando PhoneGap antes de pasar a aprender cómo usar las 
herramientas relacionadas con PhoneGap para alojar, desarrollar, probar, depurar, compilar y lanzar 
bibliotecas de aplicaciones móviles. A continuación, profundizará en el núcleo del desarrollo de la 
aplicación PhoneGap aprendiendo a usar CSS3, HTML5, JavaScript/Framework y la API de PhoneGap desde la 
perspectiva de una aplicación móvil. Con la ayuda de ejemplos paso a paso, código fácilmente comprensible y 
demostraciones dinámicas, podrá introducir soluciones móviles de extremo a extremo que integran aplicaciones 
móviles en servicios externos. Al final del curso, usted sabrá cómo utilizar PhoneGap de una manera conveniente 
para escribir aplicaciones poderosas y fascinantes que se ejecutan en cualquier lugar, rápidamente. 
Sobre el autor Keyang Xiang es el líder tecnológico y el arquitecto de aplicaciones en el departamento 
de Servicio Profesional de FeedHenry Ltd. Recibió su maestría en Ingeniería de Software de la Universidad 
de Limerick. Ha estado trabajando como ingeniero de software por más de 5 años. Sus áreas de trabajo incluyen 
principalmente soluciones de aplicaciones empresariales móviles, ingeniería de aplicaciones móviles, desarrollo 
de aplicaciones web, computación en la nube y desarrollo de soluciones empresariales basadas en Java o .
Net. Después de disfrutar de su vida adolescente en China, ahora vive en Irlanda y puede ser contactado en 
http://keyangxiang com. El empleador de Keyang, FeedHenry, proporciona una plataforma de aplicaciones 
móviles a las empresas para simplificar sus iniciativas móviles y capacitarlas para interactuar de manera más 
efectiva con los empleados, clientes y socios a través del canal móvil. La plataforma está basada en la nube y 
admite el desarrollo de aplicaciones, la integración con sistemas backend y la implementación en múltiples 
dispositivos, así como la gestión de aplicaciones y la generación de informes. ¿Cuáles son los requisitos? 
Este curso se divide en dos grandes secciones. Primero cubrimos cómo usar la plataforma PhoneGap: generar un 
binario nativo móvil que envuelve una aplicación web y luego usar las API de PhoneGap para acceder a funciones 
nativas. Luego trabajamos en optimizar sus habilidades web y discutir las mejores prácticas en el diseño de 
código, todo mientras desarrollamos una aplicación interesante en tiempo de ejecución. ¿Qué voy a obtener 
de este curso? Más de 19 conferencias y 2 horas de contenido! Obtenga eficiencia en el desarrollo de aplicaciones 
JavaScript bien estructuradas Use la herramienta Weinre para depurar y probar eficazmente las aplicaciones 
móviles de PhoneGap Implementen la comunicación del lado del servidor sin problemas utilizando AJAX y Node.js 
Hacer uso de la última Sencha Touch y jQuery Mobile frameworks para construir aplicaciones atractivas 
Comprender cómo invocar PhoneGap API y plugins de Cordova para invocar la funcionalidad nativa Maestro 
avanzado HTML5 y CSS3 para una mejor interactividad de la aplicación ¿Cuál es el público objetivo? Este 
curso es para diseñadores y desarrolladores web que desean crear aplicaciones iOS y Android utilizando 
las tecnologías que ya conocen. Este curso te ayudará a poner tus habilidades HTML, CSS y JavaScript 
para construir y enviar poderosas aplicaciones móviles. SECCIÓN 1: Inicio de su viaje Introducción a 
PhoneGapPhoneGap: Ventajas y Desventajasesección 2: Primera Aplicación Nprimer proyecto Phonegapprevisualizar 
y Desplegar un proyecto Phonegapsección 3: Configurarlo Installing the Development Tool SetDebugging 
the PhoneGap App Using ToolsUsing Git to Manage the Source CodeSECCIÓN 4: Interacciones Enriquecidas 
con PhoneGap, HTML5 y CSS3\nHTML5 and the PhoneGap AppCSS and CSS3SECCIÓN 5: The App Framework and 
JavaScript Practices Introducing the App Framworka Demo on Using a Sencha Touch Framworka Demo on 
Using a jQuery Mobile Framworkjavascript Practicesection 6: A Demo on the PhoneGap API\nUse Cordova 3 
Plugins and APIsImporting Third-Party Plugins to Cordova 2 ProjectsSECTION 7: Servers and Making the Mobile 
Application Powerful Introduction to the Server-side StackA Demo on Using AJAX to Communicate with the 
Node.JS ServerHost Server-side Stack en CloudSECTION 8: Un Ejemplo completo Demo en una Aplicación de Chat Simple'''

In [506]:
df.loc[22710,'desc'] = text

In [533]:
df[df.id == 22710]['desc'].values

array([' Herramientas tecnológicas Este curso esta diseñado para entender porque en la actualidad es necesario partir de una adecuación didáctica y trabajar desde una flexibilización tecnológica. En los videos incluidos en este curso entenderemos que el aprendizaje en la red, hoy es conectado, participativo y social, aunque exige un desarrollo de estrategias personales de organización, gestión de la información y acceso al conocimiento. El objetivo de este curso es entender que enseñar no es construir contenidos, ni transmitir información, sino crear entornos donde aprender sea posible y estimulante. What are the requirements? Personas interesadas en generar estrategias didácticas para su aplicación en el aula Personas que dessen generar un entorno de aprendizaje exitoso What am I going to get from this course? Over 17 lectures and 52 mins of content! Crear un entorno exitoso de aprendizaje Generar estrategias para trabajar en el aula de manera didáctica Adquirir una flexibilización te

## Снова  найдем похожие курсы

In [517]:
result_dic2 = {}
for lang in lang_lst:
    # Список ид курсов к которым нужно рекомендовать
    rec_id = df_rec_for2[df_rec_for2.lang == lang].id.tolist()
    
    # Языковой датафрейм со сброшенными индексами
    df_lang = df[df.lang == lang].reset_index()
    
    # Строки целевых курсов в датафрейме
    rec_row_num = df_lang[df_lang.id.isin(rec_id)].index.values
    
    # Зададим векторайзер
    vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'))
    
    # Создадим матрицу TF-IDF
    matrix = vectorizer.fit_transform(df_lang.desc)
    
    # Измерим косинусные расстояния
    cos_m  = cosine_similarity(matrix, matrix[rec_row_num])
    
    # Создадим df с расстояниями
    df_cos = pd.DataFrame(cos_m)
    
    # Добавим его к df с данными
    result_dic2['df_result' + lang] = df_lang.merge(df_cos, how='left', left_index=True, right_index=True)

## Выберем ТОП10

In [518]:
top_21040 = (
        result_dic2["df_resultru"]
            .query('id != 21040')
            .sort_values(by=[0, 'name', 'id'], ascending=(False, True, True))
            .head(10)
            .id
            .tolist()
          )

top_26676 = (
        result_dic2["df_resultru"]
            .query('id != 26676')
            .sort_values(by=[1, 'name', 'id'], ascending=(False, True, True))
            .head(10)
            .id
            .tolist()
          )

In [519]:
top_3567 = (
        result_dic2["df_resulten"]
            .query('id != 3567')
            .sort_values(by=[0, 'name', 'id'], ascending=(False, True, True))
            .head(10)
            .id
            .tolist()
          )

top_11370 = (
        result_dic2["df_resulten"]
            .query('id != 11370')
            .sort_values(by=[1, 'name', 'id'], ascending=(False, True, True))
            .head(10)
            .id
            .tolist()
          )

In [520]:
top_4176 = (
        result_dic2["df_resultes"]
            .query('id != 4176')
            .sort_values(by=[0, 'name', 'id'], ascending=(False, True, True))
            .head(10)
            .id
            .tolist()
          )

top_22710 = (
        result_dic2["df_resultes"]
            .query('id != 22710')
            .sort_values(by=[1, 'name', 'id'], ascending=(False, True, True))
            .head(10)
            .id
            .tolist()
          )

## Выгрузим результат

In [524]:
output_dic2 = {
    "21040": top_21040,
    "26676": top_26676, 
    "3567": top_3567,
    "11370": top_11370,
    "4176": top_4176,
    "22710": top22710_04,
}

In [523]:
top22710_04 = [19330, 10447, 16488, 17200, 160, 468, 23357, 24221, 21337, 22284]

In [525]:
output_dic2

{'21040': [871, 13479, 21987, 767, 924, 18772, 22055, 25830, 21421, 21042],
 '26676': [18331, 1236, 1233, 1001, 8212, 1328, 20356, 802, 20317, 807],
 '3567': [13798, 3095, 26723, 3566, 2286, 3016, 3226, 11364, 195, 524],
 '11370': [11600, 10389, 12126, 25820, 25819, 16906, 4638, 5199, 25816, 12820],
 '4176': [10160, 3660, 8098, 23350, 21400, 7944, 20404, 23629, 16929, 6864],
 '22710': [19330, 10447, 16488, 17200, 160, 468, 23357, 24221, 21337, 22284]}

In [526]:
with open('../lab07s.json', 'w') as outfile:
    json.dump(output_dic2, outfile)